# Content-based recommendation

# Exercise 1
Based on the TF-IDF vectors obtained in the Exercise 2 from Session 4, represent each user in the same vector space. Amongst other feasible solutions, you can represent a user (user profile) by computing the weighted mean of the items vectors. Compute the cosine similarity for user 'A39WWMBA0299ZF' and all products in the training set not rated by the user. What are the top-5 recommended items for user 'A39WWMBA0299ZF'? Print out the top-5 items and their similarity score.  

In [134]:
import pickle5 as pickle
import pandas as pd
import numpy as np

with open('tfidf', 'rb') as f:
    arr = np.load(f)

arr.shape

(84, 426)

In [135]:
tf = pd.read_pickle("asin_tfidf.pkl")

df = pd.read_pickle("train.pkl")
df = df[["overall", "reviewerID", "asin"]]

tf

,asin,0,1,2,3,4,5,6,7,8,...,416,417,418,419,420,421,422,423,424,425
0,B0000530HU,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1,B00006L9LC,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,B00021DJ32,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
3,B0002JHI1I,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.324781,0.0,0.0,0.000000
4,B0006O10P4,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.782655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,B019LAI4HU,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
80,B019V2KYZS,0.44916,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
81,B01BNEYGQU,0.00000,0.0,0.0,0.0,0.0,0.0,0.298296,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
82,B01DKQAXC0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.226356,0.0,0.0,0.000000,0.0,0.0,0.000000


In [136]:
# https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe
def weighted_average(df,drop_cols,weight_col,by_col):
    df['_data_times_weight'] = df.drop(drop_cols,axis=1)*df[weight_col]
    df['_weight_where_notnull'] = df[weight_col]*pd.notnull(df.drop(drop_cols,axis=1))
    g = df.groupby(by_col)
    result = g['_data_times_weight'].sum() / g['_weight_where_notnull'].sum()
    del df['_data_times_weight'], df['_weight_where_notnull']
    return result


In [137]:
full = df.merge(tf, on="asin", how="right").dropna().reset_index()
mask = ~full.columns.isin(["index", "overall", "reviewerID", "asin"])
mask2 = ~full.columns.isin(["index", "overall", "asin"])
full.loc[:, mask]  = full.loc[:, mask].mul(full["overall"], axis=0)
full = full.loc[:, mask2].groupby("reviewerID").mean()
full


# wm = lambda x: np.average(x, weights=df.loc[x.index, "overall"])
# full.drop("asin",axis=1).groupby("reviewerID").apply(lambda x: wm(x))
# weighted_average(full, ["asin", "overall"], "overall", "reviewerID")

,0,1,2,3,4,5,6,7,8,9,...,416,417,418,419,420,421,422,423,424,425
reviewerID,,,,,,,,,,,,,,,,,,,,,
A105A034ZG9EHO,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.455332,0.0,1.015447,0.0,0.0
A10JB7YPWZGRF4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.455332,0.0,0.000000,0.0,0.0
A10M2MLE2R0L6K,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
A10P0NAKKRYKTZ,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.455332,0.0,0.000000,0.0,0.0
A10ZJZNO4DAVB,0.0,0.0,0.0,0.0,0.710053,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.455332,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZCOSCQG73JZ1,0.0,0.0,0.0,0.0,0.710053,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.455332,0.0,0.000000,0.0,0.0
AZD3ON9ZMEGL6,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.455332,0.0,0.000000,0.0,0.0
AZFYUPGEE6KLW,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.455332,0.0,0.000000,0.0,0.0


In [195]:
from sklearn.metrics.pairwise import cosine_similarity

u = "A39WWMBA0299ZF"

tff = tf.copy()
tff["sim"] =  cosine_similarity(tff.drop("asin",axis=1), full[full.index == u])
# tff.drop("asin",axis=1) @ full[full.index == u].T / (np.linalg.norm(tff.drop("asin",axis=1))*np.linalg.norm(full[full.index == u].T))
tff[["asin", "sim"]].nlargest(5, "sim")

,asin,sim
78,B019FWRG3C,0.881101
39,B001LNODUS,0.881101
75,B00W259T7G,0.293022
23,B0010ZBORW,0.216363
63,B00IJHY54S,0.165912


# Exercise 2



Compute the systems’ hit rate based on the top-5, top-10 and top-20 recommendations, averaged over the total number of users. Remember that, as we are evaluating the system, you should compute the hit rate over the test set. How well/bad does this Content-based approach perform compared to the Collaborative Filtering?

# Exercise 3

Repeat Exercise 1 and 2, this time representing the products and users in a word2vec vector space. You may use the gensim library and download the 300-dimension embeddings from Google. Source: https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models

In [ ]:
import gensim.downloader
word2vec_vectors = gensim.downloader.load('word2vec-google-news-300')

